# Imports

In [1]:
import pyxdf
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import wave

# Load Data

In [2]:
xdf_path = "C:/Users/Admin/Documents/CurrentStudy/sub-P005/ses-S001/mobi/sub-P005_ses-S001_task-Story_run-001_mobi.xdf"

data, header = pyxdf.load_xdf(xdf_path)
streams_collected = [stream['info']['name'][0] for stream in data]
streams_collected

Stream 2: Calculated effective sampling rate 119.9958 Hz is different from specified rate 90.0000 Hz.
Stream 6: Calculated effective sampling rate 249.9915 Hz is different from specified rate 500.0000 Hz.


['Stimuli_Markers',
 'Tobii',
 'OpenSignals',
 'WebcamStream',
 'EGI NetAmp 0',
 'Audio']

# Stimulus

## Stimulus Trigger Duration

In [3]:
stim_dat = data[0]
stim_df = pd.DataFrame(stim_dat['time_series'])
stim_df.rename(columns={0: 'trigger'}, inplace=True)

In [4]:
stim_dat = data[0]
stim_df = pd.DataFrame(stim_dat['time_series'])
stim_df.rename(columns={0: 'trigger'}, inplace=True)
events = {
    100: 'Onset_10second_rest',
    101: 'Offset_10second_rest', 
    10: 'Onset_BirthMother',
    11: 'Offset_BirthMother', 
    20: 'Onset_CampFriend',
    21: 'Offset_CampFriend',
    30: 'Onset_FrogDissection',
    31: 'Offset_FrogDissection',
    40: 'Onset_DanceContest',
    41: 'Offset_DanceContest',
    50: 'Onset_ZoomClass',
    51: 'Offset_ZoomClass',
    60: 'Onset_Tornado',
    61: 'Offset_Tornado',
    70: 'Onset_BirthdayParty',
    71: 'Offset_BirthdayParty',
    80: 'Onset_Glitter',
    81: 'Offset_Glitter',
}

# relabel the event if the trigger is in the events dictionary, else if 
stim_df['event'] = stim_df['trigger'].apply(lambda x: events[x] if x in events.keys() else 'Bx_input')

# relabel the event as a psychopy timestamp if the trigger is greater than 4 digits
stim_df.loc[stim_df.trigger.astype(str).str.len() > 4, 'event'] = 'psychopy_time_stamp'
stim_df['lsl_time_stamps'] = stim_dat['time_stamps']
stim_df['time'] = stim_dat['time_stamps'] - stim_dat['time_stamps'][0]
stim_df.head(10)

,trigger,event,lsl_time_stamps,time
0,100,Onset_10second_rest,339430.267538,0.000000
1,1722614126,psychopy_time_stamp,339430.267550,0.000013
2,101,Offset_10second_rest,339440.284336,10.016798
3,1722614136,psychopy_time_stamp,339440.284345,10.016807
4,60,Onset_Tornado,339440.362039,10.094501
5,1722614136,psychopy_time_stamp,339440.362051,10.094513
6,61,Offset_Tornado,339603.605601,173.338063
7,1722614299,psychopy_time_stamp,339603.605611,173.338073
8,7,Bx_input,339609.753684,179.486146
9,6,Bx_input,339617.836658,187.569121


In [5]:
story_onsets = list(events.keys())[2:][::2] #every other trigger is the offset

story_onsets

[10, 20, 30, 40, 50, 60, 70, 80]

In [6]:
def get_secs_between_triggers(trigger1, trigger2):
    return stim_df.loc[stim_df.trigger == trigger1, 'time'].values[0] - stim_df.loc[stim_df.trigger == trigger2, 'time'].values[0]

# Get a list of the story names using the events dict
story_names = [x.split('_')[1] for x in list(events.values())[2:][::2]]
story_names

['BirthMother',
 'CampFriend',
 'FrogDissection',
 'DanceContest',
 'ZoomClass',
 'Tornado',
 'BirthdayParty',
 'Glitter']

In [7]:
files= [
    "../audio_files/audio_files/After_Searching_I_Found_and_Met_My_Birth_Mother.wav",
    "../audio_files/audio_files/Camp_Lose_A_Friend.wav",
    "../audio_files/audio_files/Frog_Dissection_Disaster.wav",
    "../audio_files/audio_files/I_Decided_To_Be_Myself_And_Won_A_Dance_Contest.wav",
    "../audio_files/audio_files/I_Fully_Embarrassed_Myself_In_Zoom_Class1.wav",
    "../audio_files/audio_files/Left_Home_Alone_in_a_Tornado.wav",
    "../audio_files/audio_files/The_Birthday_Party_Prank.wav",
    "../audio_files/audio_files/My_Dad_Saw_Me_Wearing_Glitter_At_School.wav"
]


durations = pd.DataFrame({
    'trigger':story_onsets,
    'story':[events[x] for x in story_onsets],
    'lsl_duration': [get_secs_between_triggers(x+1, x) for x in story_onsets],
    'audiofile_duration': [wave.open(x).getnframes()/wave.open(x).getframerate() for x in files]
})

durations['difference(sec)'] = durations['audiofile_duration'] - durations['lsl_duration']
durations

,trigger,story,lsl_duration,audiofile_duration,difference(sec)
0,10,Onset_BirthMother,157.745044,157.726104,-0.018939
1,20,Onset_CampFriend,152.937353,152.838063,-0.099290
2,30,Onset_FrogDissection,124.361157,124.344813,-0.016344
3,40,Onset_DanceContest,138.335716,138.318729,-0.016987
4,50,Onset_ZoomClass,98.378069,98.359854,-0.018215
5,60,Onset_Tornado,163.243562,163.227583,-0.015979
6,70,Onset_BirthdayParty,160.231821,160.215313,-0.016509
7,80,Onset_Glitter,158.391881,158.380208,-0.011673


## Stimulus wav file Duration

In [8]:
def get_wave_duration(file_path):
    with wave.open(file_path, 'rb') as wave_file:
        frame_rate = wave_file.getframerate()
        n_frames = wave_file.getnframes()
        duration = n_frames / frame_rate
        return duration
    

tornado_dur = get_wave_duration("../audio_files/audio_files/Left_Home_Alone_in_a_Tornado.wav")
tornado_dur

163.22758333333334